In [1]:
import os
import json
import pandas as pd

In [ ]:
# fold = r"../data"

# data = []

# for file in os.listdir(fold):
#     if not file.startswith("新一代信息技术产业集群"):
#         continue
#     file_name = os.path.join(fold, file)
#     data.append(pd.read_excel(file_name))

# df = pd.concat(data)
# df.to_csv("../data/2.5w_新一代信息技术企业.csv", index=False)

In [2]:
df = pd.read_csv("../data/2.5w_新一代信息技术企业.csv")
df.shape

(24451, 38)

In [3]:
def load_json(file):
    with open(file, "r", encoding="utf-8") as f:
        return json.load(f)

In [5]:
baidu_map_json_data = load_json("API_results/企业注册地址.json")

In [6]:
def get_lng_lat(address):
    res = baidu_map_json_data.get(address, {})
    if not res:
        return pd.NA, pd.NA
    obj = res["result"]["location"]
    return obj["lng"], obj["lat"]

In [7]:
get_lng_lat("合肥高尔生命健康科学研究院有限公司")

(<NA>, <NA>)

In [8]:
baidu_map_json_data

{'宜兴市官林镇戈庄村': {'status': 0,
  'result': {'location': {'lng': 119.69968771208765,
    'lat': 31.525741491744384},
   'precise': 0,
   'confidence': 50,
   'comprehension': 100,
   'level': '村庄'},
  'idx': 3679,
  'address_input': '宜兴市官林镇戈庄村'},
 '广州市黄埔区松岗环北路二巷8号101房': {'status': 0,
  'result': {'location': {'lng': 113.48707083968644,
    'lat': 23.187491541424553},
   'precise': 0,
   'confidence': 20,
   'comprehension': 100,
   'level': '区县'},
  'idx': 12337,
  'address_input': '广州市黄埔区松岗环北路二巷8号101房'},
 '武汉经济技术开发区（汉南区）通江二路哈工大机器人武汉产业基地项目研发楼051号': {'status': 0,
  'result': {'location': {'lng': 114.07229174145935,
    'lat': 30.295057069925097},
   'precise': 0,
   'confidence': 30,
   'comprehension': 35,
   'level': '道路'},
  'idx': 20995,
  'address_input': '武汉经济技术开发区（汉南区）通江二路哈工大机器人武汉产业基地项目研发楼051号'},
 '四川省成都高新区天映路136号': {'status': 0,
  'result': {'location': {'lng': 103.91945476251473,
    'lat': 30.770296500740436},
   'precise': 1,
   'confidence': 80,
   'comprehension': 100,
   'leve

## 添加经纬度

In [15]:
def add_lng_lat(row):
    # company_name = row["企业名称"]
    address = row["注册地址"]
    if isinstance(address, str):
        address = address.strip()
    lng, lat = get_lng_lat(address)
    return pd.Series({"经度": lng, "纬度": lat})

In [16]:
df.apply(add_lng_lat, axis=1).shape

(24451, 2)

In [17]:
df.shape

(24451, 40)

In [18]:
df[["经度", "纬度"]] = df.apply(add_lng_lat, axis=1)

In [13]:
df.shape

(24451, 40)

In [19]:
df.to_csv("../output/2.5w_新一代信息技术企业经纬度_by_address.csv", index=False)

In [ ]:
# df.to_excel("../output/2.5w_新一代信息技术企业经纬度.xlsx", index=False)